In [ ]:
# Elasticsearch 




In [ ]:
import requests
import pandas as pd

# Elasticsearch base URL
base_url = "http://192.168.20.106:9200"
# Index name
index = "winlogbeat-*"

# Initial search request to start scrolling
initial_response = requests.post(
    f"{base_url}/{index}/_search?scroll=1m",
    json={
        "size": 10000,  # Adjust the size as per your requirement
        "query": {"match_all": {}}
    }
).json()

# Extract scroll ID from the initial response
scroll_id = initial_response["_scroll_id"]

# Process initial batch of documents
hits = initial_response["hits"]["hits"]
data = [hit["_source"] for hit in hits]

# Track total documents retrieved
total_documents_retrieved = len(data)
print(f"Retrieved {total_documents_retrieved} documents.")

# Loop to fetch subsequent batches of documents until no more documents are left
while hits:
    # Fetch next batch of documents using scroll API
    response = requests.post(
        f"{base_url}/_search/scroll",
        json={"scroll": "1m", "scroll_id": scroll_id}
    ).json()
    
    # Extract scroll ID from the response
    scroll_id = response["_scroll_id"]
    
    # Process batch of documents
    hits = response["hits"]["hits"]
    
    # If no hits, break out of the loop
    if not hits:
        break
    
    # Extend data with new batch of documents
    data.extend([hit["_source"] for hit in hits])
    
    # Update total documents retrieved
    total_documents_retrieved += len(hits)
    print(f"Retrieved {total_documents_retrieved} documents.")

# Convert data to pandas DataFrame
df = pd.DataFrame(data)

# Display DataFrame
print(df)


In [21]:
df.to_json("lab_logs_normal_activity_may_6_2024.json")